In [67]:
import pandas as pd
import xlwings as xw
from pathlib import Path
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle

In [68]:
import os
import time
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import stats
from pylab import rcParams
from sklearn.utils import check_random_state
# from sklearn.datasets import load_digits
# from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import confusion_matrix
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, \
GradientBoostingClassifier

max_queried = 500
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html

#### Unlocking files and saving it in the local

In [69]:
# def unlock_file(data_file, file_nm):
#     path = '../data'
#     workbook = xw.Book(os.path.join(path, data_file))
#     df = workbook.sheets[0].range('A1').options(pd.DataFrame, 
#                                             header=1,
#                                             index=False, 
#                                             expand='table').value
#     new_dir = "../data/extract"
#     Path().mkdir(parents=True, exist_ok=True)
#     df.to_csv(os.path.join(new_dir, file_nm), index =False, header= True)


In [70]:
# data_file = 'persona_output.xlsx'
# file_nm = 'full_output.csv'
# unlock_file(data_file, file_nm) 

In [79]:
pth_1 = '../data/extract/p_labels.csv'
pth_2  = '../data/extract/full_output.csv'
df1 = pd.read_csv(pth_1)
df2 = pd.read_csv(pth_2)
df2 = df2[~df2['grpcis'].str.contains('[a-zA-Z]')]
df2['grpcis'] = df2['grpcis'].astype(float)
df2['customer_no'] = df2['customer_no'].astype(float)
display(df1.info())
display(df2.info())
df_f  = pd.merge(df2, df1[['grpcis', 'customer_no', 'director_name', 'label']], how='left', left_on=['grpcis', 'customer_no', 'director_name'], right_on=['grpcis', 'customer_no', 'director_name'])
display(df_f.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7150 entries, 0 to 7149
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   grpcis              7150 non-null   float64
 1   customer_no         7150 non-null   float64
 2   director_name       7150 non-null   object 
 3   match-komal         7150 non-null   object 
 4   passport_number     4436 non-null   object 
 5   visa_no             2368 non-null   object 
 6   mobile_1            2296 non-null   float64
 7   mobile_2            1688 non-null   float64
 8   national_id_number  19 non-null     float64
 9   Age                 5250 non-null   float64
 10  Group               7150 non-null   object 
 11  fuzzy_grp_nms       7150 non-null   object 
 12  label               7150 non-null   bool   
dtypes: bool(1), float64(6), object(6)
memory usage: 677.4+ KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56858 entries, 0 to 56858
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   grpcis              56858 non-null  float64
 1   customer_no         56858 non-null  float64
 2   director_name       56853 non-null  object 
 3   passport_number     36384 non-null  object 
 4   visa_no             21074 non-null  object 
 5   mobile_1            23022 non-null  float64
 6   mobile_2            15776 non-null  float64
 7   e_mail              14061 non-null  object 
 8   national_id_number  211 non-null    object 
 9   age                 40440 non-null  float64
 10  id                  56848 non-null  float64
 11  group               56849 non-null  object 
 12  fuzzy_grp_nms_f     56847 non-null  object 
 13  dob                 4 non-null      object 
dtypes: float64(6), object(8)
memory usage: 6.5+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56858 entries, 0 to 56857
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   grpcis              56858 non-null  float64
 1   customer_no         56858 non-null  float64
 2   director_name       56853 non-null  object 
 3   passport_number     36384 non-null  object 
 4   visa_no             21074 non-null  object 
 5   mobile_1            23022 non-null  float64
 6   mobile_2            15776 non-null  float64
 7   e_mail              14061 non-null  object 
 8   national_id_number  211 non-null    object 
 9   age                 40440 non-null  float64
 10  id                  56848 non-null  float64
 11  group               56849 non-null  object 
 12  fuzzy_grp_nms_f     56847 non-null  object 
 13  dob                 4 non-null      object 
 14  label               6567 non-null   object 
dtypes: float64(6), object(9)
memory usage: 6.9+ MB


None

In [80]:
cls = ['grpcis', 'customer_no', 'director_name', 'fuzzy_grp_nms_f', 'label']
df1_new = df_f[cls]
df1_new.replace({False:0, True:1}, inplace=True)
display(df1_new.info())
display(df1_new.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56858 entries, 0 to 56857
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   grpcis           56858 non-null  float64
 1   customer_no      56858 non-null  float64
 2   director_name    56853 non-null  object 
 3   fuzzy_grp_nms_f  56847 non-null  object 
 4   label            6567 non-null   float64
dtypes: float64(3), object(2)
memory usage: 2.6+ MB


None

,grpcis,customer_no,director_name,fuzzy_grp_nms_f,label
0,13157197.0,13157181.0,JONATHAN PETER MCGLOIN,JONATHAN PETER MCGLOIN,NaN
1,13157197.0,13157181.0,MELANIE MARIA MENEZES FERNANDES,MELANIE MARIA MENEZES FERNANDES,NaN
2,13157197.0,13157789.0,MUHAMMAD ASIF SIDDIQUE,MUHAMMAD ASIF SIDDIQUE,NaN
3,13157197.0,13157181.0,ZAMAN KOCHU KUNJU,ZAMAN KOCHU KUNJU,NaN
4,13157197.0,13156069.0,RAMAPURATH AJITKUMAR MENON,RAMAPURATH AJITKUMAR MENON,NaN


In [81]:
## Spliting the data
X, y = df1_new[['grpcis', 'customer_no', 'director_name', 'fuzzy_grp_nms_f']], df1_new[['label']]

In [82]:
X_train_full,X_test,y_train_full,y_test=train_test_split(X,y,test_size=0.15,random_state=123)

In [83]:
## An experiment with SVM
inital_lb_samp =  250
X_full_shape = X_train_full.shape[0]
print(X_full_shape)

48329


In [84]:
display(df1_new.shape)
df1_n1 =  df1_new.dropna(subset=['label'], how = 'all')
display(df1_n1.shape)

(56858, 5)

(6567, 5)

In [91]:
## Spliting the data
X_lab, y_lab = df1_n1[['grpcis', 'customer_no', 'director_name', 'fuzzy_grp_nms_f']], df1_n1[['label']]
# X_train_full_lab, X_test_lab, y_train_full_lab, y_test_lab=train_test_split(X,y,test_size=0.15,random_state=123)

In [92]:
X_lab.shape[0]

6567

### initialize process by applying base learner to labeled training data set to obtain Classifier

In [98]:
random_state = check_random_state(0)

permutation = np.random.choice(X_lab.shape[0],
                                inital_lb_samp,
                                replace=False)
print('initial random chosen samples', permutation.shape)

X_train = X_lab.iloc[permutation]
# print('Training features shape', X_train)
y_train = y_lab.iloc[permutation]
# print('Test features shape', X_train)
print(y_train.value_counts())
X_train = X_train.values
y_train = y_train.values
X_train = X_train.reshape((X_train.shape[0], -1))

y_train = y_train[:,0]
print(y_train.shape)
print(X_train.shape)
bin_count = np.bincount(y_train.astype('int64'))
unique = np.unique(y_train.astype('int64'))

print (
    'initial train set:',
    X_train.shape,
    y_train.shape,
    'unique(labels):',
    bin_count,
    unique
    )



initial random chosen samples (250,)
label
1.0      217
0.0       33
dtype: int64
(250,)
(250, 4)
initial train set: (250, 4) (250,) unique(labels): [ 33 217] [0 1]


In [99]:
queried  = inital_lb_samp
samplecount = [inital_lb_samp]
print(f'Queried: {queried}\t samplecount: {samplecount}')

Queried: 250	 samplecount: [250]


### assign the val set the rest of the 'unlabelled' training data

In [104]:
X_val = np.array([])
y_val = np.array([])
X_val = np.copy(X)
X_val = np.delete(X_val, permutation, axis=0)

y_val = np.copy(y)
y_val = np.delete(y_val, permutation, axis=0)

print ('val set:', X_val.shape, y_val.shape, permutation.shape)

val set: (56608, 4) (56608, 1) (250,)


(56608, 4) (56858, 4)


(56608, 1) (56858, 1)


val set: (6900, 4) (6900, 1) (250,)

